In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Lambda, Normalize, CenterCrop
from torchvision.io.image import ImageReadMode
from torch import nn
import torchvision.models as models

from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
!pip install import-ipynb
import import_ipynb


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os


GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = None
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', 'image_color','image_color')
print(os.listdir(GOOGLE_DRIVE_PATH))

['README.md', '0.jpg', '.ipynb_checkpoints', 'train_landscape_images', 'landscape_images', 'train_mine.ipynb', 'basic_model_mine.ipynb', '__pycache__', 'colorize_mine.ipynb', 'basic_model.py', 'colorize_data.py', 'best.pt', 'train.py', 'test.ipynb', 'main.ipynb']


In [6]:
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

In [7]:
import test
import colorize_data
import basic_model
import train
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
import torch

In [8]:
input_transform = T.Compose([T.ToTensor(),
                                          T.Resize(size=(256,256)),
                                          T.Grayscale(),
                                          T.Normalize((0.5), (0.5))
                                          ])
        # Use this on target images(colorful ones)
target_transform = T.Compose([T.ToTensor(),
                                           T.Resize(size=(256,256)),
                                           T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [9]:
#model_det = colorize_data.ColorizeData()
dataset = colorize_data.ColorizeData()
k = len(dataset)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [3000,1281])
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)

/content/drive/MyDrive/image_color/image_color/landscape_images


In [10]:
epoch_curr=0

In [12]:
import train
FILE = './checkpoints_25.pth'
#epoch_curr = 0
#device 
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=0.0)
train_obj = train.Trainer(1e-2,10)
loss_best = 1e10
for epoch in range(20):
  print("Epoch number",epoch + epoch_curr+1)
  # Train for one epoch, then validate
  train_obj.train()
  with torch.no_grad():
    losses,model,optimizer = train_obj.validate()
  if losses < loss_best:
        loss_best = losses
        path_save = os.path.join(GOOGLE_DRIVE_PATH, 'best_25.pt')
        #model.save(path_save)
        torch.save(model.state_dict(), path_save)
  checkpoint = {
      "epoch": epoch + epoch_curr+ 1,
      "model_state": model.state_dict(),
      "optim_state": optimizer.state_dict()
      }
  torch.save(checkpoint , FILE)

/content/drive/MyDrive/image_color/image_color/landscape_images
Epoch number 1
loss: 0.444623  [    0/ 3000]
loss: 1.292895  [   64/ 3000]
loss: 0.496906  [  128/ 3000]
loss: 0.562007  [  192/ 3000]
loss: 0.419917  [  256/ 3000]
loss: 0.381925  [  320/ 3000]
loss: 0.354276  [  384/ 3000]
loss: 0.346747  [  448/ 3000]
loss: 0.280459  [  512/ 3000]
loss: 0.253587  [  576/ 3000]
loss: 0.215735  [  640/ 3000]
loss: 0.191188  [  704/ 3000]
loss: 0.185256  [  768/ 3000]
loss: 0.192855  [  832/ 3000]
loss: 0.165474  [  896/ 3000]
loss: 0.186686  [  960/ 3000]
loss: 0.145396  [ 1024/ 3000]
loss: 0.142490  [ 1088/ 3000]
loss: 0.129177  [ 1152/ 3000]
loss: 0.114037  [ 1216/ 3000]
loss: 0.119760  [ 1280/ 3000]
loss: 0.122424  [ 1344/ 3000]
loss: 0.123820  [ 1408/ 3000]
loss: 0.095628  [ 1472/ 3000]
loss: 0.117711  [ 1536/ 3000]
loss: 0.113409  [ 1600/ 3000]
loss: 0.096578  [ 1664/ 3000]
loss: 0.119473  [ 1728/ 3000]
loss: 0.114663  [ 1792/ 3000]
loss: 0.094284  [ 1856/ 3000]
loss: 0.107480  [ 192

KeyboardInterrupt: ignored

In [ ]:
##optional bloack made to load checkpoints if training stops in between. Done to overcome cell run time limit on Colab.
loaded_checkpoint = torch.load("./checkpoints_25.pth")
epoch_curr = loaded_checkpoint["epoch"]
print(epoch_curr)

model = basic_model.Net()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

model.load_state_dict(loaded_checkpoint["model_state"])
optimizer.load_state_dict(loaded_checkpoint["optim_state"])
